# Importing libraies

In [1]:
import random
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Embedding, Input, Bidirectional, GlobalMaxPooling1D, Attention, BatchNormalization, Flatten
from tensorflow.keras.optimizers import RMSprop, Adam
from numba import jit, cuda
import re
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
import keras.utils as ku
from keras.callbacks import EarlyStopping
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

def remove_four_spaces(line):
    new_line = ""
    for idx, char in enumerate(line):
        if char == ' ' and idx + 3 < len(line) and line[idx + 1] == ' ' and line[idx + 2] == ' ':
            new_line += line[idx + 4:]
            break
    return new_line

def preprocess_text(text):
    patterns = [('+', ''), ('$', ''), ('murderland', ''), ('\d+', '')]
    for old, new in patterns:
        text = text.replace(old, new)
    lines = text.splitlines()
    new_text_lines = []
    for line in lines:
        new_line = remove_four_spaces(line)
        new_text_lines.append(new_line)
    new_text = "\n".join(new_text_lines)
    new_text = new_text.replace('\n\n', "")
    new_text = new_text.translate(str.maketrans("", "", punctuation))
    return new_text.lower()

def load_script(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        script = f.read()
    return script

def remove_empty_lines(text):
    lines = text.split("\n")
    cleaned_lines = [line.strip() for line in lines if line.strip()]
    cleaned_text = "\n".join(cleaned_lines)
    return cleaned_text

def remove_numbers(text):
    cleaned_text = re.sub(r'\d+', '', text)
    return cleaned_text

def remove_uppercase_with_spaces_or_adjacent_and_spaces_before_first_word(text):
    pattern = r'(?:\s{3,})([A-Z]+)(?:\s{3,})|([A-Z]+)(?:\s{3,})|(?:\s{3,})([A-Z]+\s[A-Z]+)'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text


In [ ]:

# Load scripts
#movie_quotes_script = load_script("/content/drive/MyDrive/moviequotes.scripts.txt")
pulp_fiction_script = load_script("/content/drive/MyDrive/Pulp Fiction script.txt")
Stephen_King_script = load_script("/content/drive/MyDrive/Stephen King Script.log")

filtered_text = ""
with open("/content/drive/MyDrive/jocker_script.txt", "r") as file:
    # Read each line in the file
    for line_num, line in enumerate(file, start=1):
        # Check if the line contains any lowercase letters
        if any(char.islower() for char in line):
            filtered_text += line

cleaned_text = re.sub(r'\(.*?\)', '', filtered_text, flags=re.DOTALL)
cleaned_text = '\n'.join(line.lstrip() for line in cleaned_text.split('\n'))
word_to_delete = "Beat."
cleaned_text = '\n'.join(line for line in cleaned_text.split('\n') if line.strip() != word_to_delete)
cleaned_text = cleaned_text.translate(str.maketrans("", "", punctuation))
jocker_script = cleaned_text.replace("’", "").lower()
jocker_script = remove_empty_lines(jocker_script)
jocker_script = remove_numbers(jocker_script)

# Preprocess scripts
#movie_quotes_script = preprocess_text(movie_quotes_script)
#print(jocker_script)
jocker_len = len(jocker_script.split())
jocker_lenLine = len(jocker_script.splitlines())

filtered_text = ""
with open("/content/drive/MyDrive/THE SHAWSHANK REDEMPTION.txt", "r") as file:
    # Read each line in the file
    for line_num, line in enumerate(file, start=1):
        # Check if the line contains any lowercase letters
        if any(char.islower() for char in line):
            filtered_text += line

THE_SHAWSHANK_script = filtered_text.translate(str.maketrans("", "", punctuation))
THE_SHAWSHANK_script = THE_SHAWSHANK_script.replace("’", "").lower()

for lineS in THE_SHAWSHANK_script.splitlines():
  if 'int' in lineS:
    THE_SHAWSHANK_script = THE_SHAWSHANK_script.replace(lineS, "")

THE_SHAWSHANK_script = remove_empty_lines(THE_SHAWSHANK_script)
THE_SHAWSHANK_script = remove_numbers(THE_SHAWSHANK_script)
THE_SHAWSHANK_len = len(THE_SHAWSHANK_script.split())
THE_SHAWSHANK_lenLine = len(THE_SHAWSHANK_script.splitlines())



pulp_fiction_script = remove_uppercase_with_spaces_or_adjacent_and_spaces_before_first_word(pulp_fiction_script)
pulp_fiction_script = pulp_fiction_script.lower()
pulp_fiction_script = pulp_fiction_script.translate(str.maketrans("", "", punctuation))
pulp_fiction_script = pulp_fiction_script.replace("’", "")
pulp_fiction_script = remove_empty_lines(pulp_fiction_script)
pulp_fiction_script = remove_numbers(pulp_fiction_script)

pulp_len = len(pulp_fiction_script.split())
pulp_lenLine = len(pulp_fiction_script.splitlines())


Stephen_King_script = Stephen_King_script.lower()
Stephen_King_script = Stephen_King_script.translate(str.maketrans("", "", punctuation))
Stephen_King_script = Stephen_King_script.replace("’", "")
Stephen_King_script = remove_empty_lines(Stephen_King_script)
Stephen_King_script = remove_numbers(Stephen_King_script)


Stephen_King_len = len(Stephen_King_script.split())
Stephen_King_lenLine = len(Stephen_King_script.splitlines())

#print(Stephen_King_script)

text = jocker_script +'\n'  + THE_SHAWSHANK_script + '\n' + pulp_fiction_script + '\n' + Stephen_King_script
# Create tokenizer
tokenizer = Tokenizer()
corpus = text.lower().split("\n")
tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1


In [ ]:
# Create tokenizer
tokenizer = Tokenizer()
corpus = text.lower().split("\n")
tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1
# Create input sequences
input_seq = []
lenth = 0
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i + 1]
        input_seq.append(n_gram_seq)


max_seq_len = max([len(x) for x in input_seq])
input_seq = np.array(pad_sequences(input_seq, maxlen = max_seq_len, padding='pre'))
predictors, label = input_seq[:, :-1], input_seq[:, -1]

label = ku.to_categorical(label, num_classes=total_words)

# Load GloVe embeddings
# GloVe dive link : https://drive.google.com/file/d/1IhO9d0ZzQjAlZKzsYZLAzJ4-taC_ljAb/view?usp=drive_link
glove_path = '/content/drive/MyDrive/glove.6B.100d.txt'
embeddings_index = dict()
with open(glove_path, encoding="utf8") as glove:
    for line in glove:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
glove.close()

embedding_matrix = np.zeros((total_words, 100))
for word, index in tokenizer.word_index.items():
    if index > total_words - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector


In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.27085999,  0.044006  , -0.02026   , ..., -0.4923    ,
         0.63687003,  0.23642001],
       ...,
       [-0.0061811 ,  0.25497001,  0.52074999, ..., -0.28420001,
        -0.70090002,  0.047897  ],
       [ 0.47593001,  0.35716   , -0.24327999, ..., -0.097822  ,
         0.9131    , -0.18909   ],
       [-0.02838   ,  0.47933999, -0.47825   , ..., -0.45166999,
         0.10207   ,  0.099218  ]])

In [ ]:
first_trainL = label[:jocker_len - jocker_lenLine]
second_trainL = label[jocker_len - jocker_lenLine : THE_SHAWSHANK_len + jocker_len - jocker_lenLine - THE_SHAWSHANK_lenLine]
third_trainL = label[THE_SHAWSHANK_len + jocker_len - jocker_lenLine - THE_SHAWSHANK_lenLine : pulp_len + THE_SHAWSHANK_len + jocker_len - jocker_lenLine - THE_SHAWSHANK_lenLine - pulp_lenLine]
fourth_trainL = label[pulp_len + THE_SHAWSHANK_len + jocker_len - jocker_lenLine - THE_SHAWSHANK_lenLine - pulp_lenLine : ]


In [ ]:
first_trainP = predictors[:jocker_len - jocker_lenLine]
second_trainP = predictors[jocker_len - jocker_lenLine : THE_SHAWSHANK_len + jocker_len - jocker_lenLine - THE_SHAWSHANK_lenLine]
third_trainP = predictors[THE_SHAWSHANK_len + jocker_len - jocker_lenLine - THE_SHAWSHANK_lenLine : pulp_len + THE_SHAWSHANK_len + jocker_len - jocker_lenLine - THE_SHAWSHANK_lenLine - pulp_lenLine]
fourth_trainP = predictors[pulp_len + THE_SHAWSHANK_len + jocker_len - jocker_lenLine - THE_SHAWSHANK_lenLine - pulp_lenLine : ]

In [ ]:
from keras import backend as K
from keras.layers import Layer

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = K.squeeze(K.tanh(K.dot(x, self.W) + self.b), axis=-1)
        at = K.softmax(et)
        at = K.expand_dims(at, axis=-1)
        output = x * at
        return K.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [ ]:
# Create the model
model2 = Sequential()
model2.add(Embedding(total_words, 100, weights=[embedding_matrix], input_length=max_seq_len - 1))
model2.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model2.add(AttentionLayer())
model2.add(Dropout(0.2))
model2.add(Dense(total_words, activation='softmax'))
model2.summary()
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model2.fit(predictors, label, validation_split=0.2, epochs=20, batch_size=128, verbose=1)
model2.save('/content/drive/MyDrive/TextModel2.h5')



In [ ]:
# Train the model
import matplotlib.pyplot as plt
from keras.models import load_model, Model
from keras.utils import CustomObjectScope

# Define a function to load the model with custom objects
def load_model_with_custom_objects(model_path):
    with CustomObjectScope({'AttentionLayer': AttentionLayer}):
        return load_model(model_path)

# model on dive : https://drive.google.com/file/d/1-21TX3i-TKUr6MmpupCTp-61NTm0vGoR/view?usp=drive_link
model2 = load_model_with_custom_objects('/content/drive/MyDrive/TextModel2.h5')
model2.summary()
# Assuming 'model2' is your pre-trained model

# Assuming `history` is the object returned by `model.fit()`
model2.fit(predictors, label, validation_split=0.2, epochs=3, batch_size=128, verbose=1)
model2.save('/content/drive/MyDrive/TextModel2.h5')


In [ ]:
def generate_text(seed_text, next_words, max_seq_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len - 1, padding='pre')
        predicted = model2.predict(token_list, verbose=0)
        index = np.argmax(predicted[0])
        output_word = ""
        for word, idx in tokenizer.word_index.items():
            if idx == index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Train the model
import matplotlib.pyplot as plt
from keras.models import load_model, Model
from keras.utils import CustomObjectScope

# Define a function to load the model with custom objects
def load_model_with_custom_objects(model_path):
    with CustomObjectScope({'AttentionLayer': AttentionLayer}):
        return load_model(model_path)


model2 = load_model_with_custom_objects('/content/drive/MyDrive/TextModel2.h5')
model2.summary()


# Example reference and generated text
reference_text = " ".join(Stephen_King_script.split()[:25])
generated_text = generate_text(" ".join(Stephen_King_script.split()[:5]), 20, max_seq_len)

#print(f'\nGeneration :{generated_text}')
#print(f'Refrence: {reference_text}\n')

#print('='*200)


generated_text2 = generate_text('i like to walk', 20, max_seq_len)
print(f'\nGeneration :{generated_text2}')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 21, 100)           1019000   
                                                                 
 bidirectional (Bidirection  (None, 21, 512)           731136    
 al)                                                             
                                                                 
 attention_layer (Attention  (None, 512)               533       
 Layer)                                                          
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 10190)             5227470   
                                                                 
Total params: 6978139 (26.62 MB)
Trainable params: 69781

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu

def bleu(refs, gens):
    '''
    Calculate pair-wise BLEU score using NLTK implementation.

    Args:
        refs : a list of reference sentences
        gens : a list of candidate (generated) sentences

    Returns:
        bleu score (float)
    '''
    ref_bleu = []
    gen_bleu = []

    for ref, gen in zip(refs, gens):
        ref_bleu.append([ref.split()])
        gen_bleu.append(gen.split())

    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

import itertools

# Supporting function
def _split_into_words(sentences):
    """Splits multiple sentences into words and flattens the result"""
    return list(itertools.chain(*[_.split(" ") for _ in sentences]))

# Supporting function
def _get_word_ngrams(n, sentences):
    """Calculates word n-grams for multiple sentences."""
    assert len(sentences) > 0
    assert n > 0

    words = _split_into_words(sentences)
    return _get_ngrams(n, words)

# Supporting function
def _get_ngrams(n, text):
    """Calculates n-grams."""
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set

def rouge_n(reference_sentences, evaluated_sentences, n=2):
    """
    Computes ROUGE-N of two text collections of sentences.
    Args:
        evaluated_sentences: The sentences that have been picked by the summarizer.
        reference_sentences: The sentences from the reference set.
        n: Size of n-grams. Defaults to 2.
    Returns:
        Recall ROUGE score (float).
    Raises:
        ValueError: Raises exception if a param has len <= 0.
    """
    if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
        raise ValueError("Collections must contain at least 1 sentence.")

    evaluated_ngrams = _get_word_ngrams(n, evaluated_sentences)
    reference_ngrams = _get_word_ngrams(n, reference_sentences)
    reference_count = len(reference_ngrams)
    evaluated_count = len(evaluated_ngrams)

    # Gets the overlapping n-grams between evaluated and reference
    overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
    overlapping_count = len(overlapping_ngrams)

    # Handle edge case. This isn't mathematically correct, but it's good enough
    if evaluated_count == 0:
        precision = 0.0
    else:
        precision = overlapping_count / evaluated_count

    if reference_count == 0:
        recall = 0.0
    else:
        recall = overlapping_count / reference_count

    # F1-score is not needed, just returning recall count in ROUGE, useful for our purpose
    return recall




In [ ]:
bleu_score = bleu(reference_text, generated_text)
recall_score1= rouge_n([reference_text], [generated_text], 1)
recall_score2 = rouge_n([reference_text], [generated_text], 2)

print("BLEU Score:", bleu_score)
print("ROUGE Recall Score 1:", recall_score1)
print("ROUGE Recall Score 2:", recall_score2)


BLEU Score: 0.0034871578252686666
ROUGE Recall Score 1: 0.4117647058823529
ROUGE Recall Score 2: 0.2631578947368421
